
%md

# Construção das Dimensões Analíticas – Modelo Dimensional DEX

Este notebook tem como objetivo a criação das tabelas dimensionais que compõem o modelo estrela derivado da camada Bronze (`workspace.ethereum.dex_trades_bronze`). A construção dessas dimensões permite uma organização semântica e otimizada dos dados, facilitando a criação de joins eficientes e análises analíticas consistentes.

---

## Objetivo

Cada dimensão representa uma entidade desnormalizada da realidade de transações em exchanges descentralizadas, como tokens, redes, exchanges, contratos e datas. As dimensões são persistidas em formato Delta e servirão de base para a criação da tabela fato `fato_dex_transacoes`.

---

## Dimensão `dim_calendario`

### Motivação

A dimensão calendário foi criada a partir dos dias distintos registrados na coluna `data_transacao`. É fundamental para análise temporal, agregações por mês, dia da semana e identificação de fins de semana.

### Etapas realizadas

- Extração dos dias únicos da camada Bronze  
- Cálculo de colunas derivadas como:  
  - `id_data` no formato `yyyyMMdd`  
  - Nome do mês e do dia traduzidos para português  
  - Sinalização se o dia é fim de semana (sábado ou domingo)  
- Utilização de expressões `create_map` para mapear nomes de meses e dias em inglês para português  
- Escrita da dimensão `dim_calendario` com partições otimizadas  

---

## Dimensão `dim_exchange`

### Motivação

Essa dimensão representa as exchanges descentralizadas envolvidas nas transações. A existência de nomes distintos e protocolos exige uma estrutura padronizada com chave substituta.

### Etapas realizadas

- Extração de campos distintos: `exchange_nome`, `exchange_nome_completo`, `protocolo`  
- Aplicação de funções de limpeza (`lower`, `trim`, `coalesce`)  
- Geração de chave substituta (`id_exchange`) usando função hash `sha2` baseada na combinação dos campos  
- Escrita em tabela Delta como `dim_exchange`  

---

## Dimensão `dim_token`

### Motivação

Tokens aparecem nas transações como moeda base ou moeda de cotação. Esta dimensão unifica os dois contextos para consolidar todos os tokens presentes no histórico.

### Etapas realizadas

- Seleção dos tokens base (`moeda_base`, `nome_base`, `endereco_base`) e quote (`moeda_quote`, `nome_quote`, `endereco_quote`)  
- União das duas fontes em um único DataFrame sem duplicidade  
- Geração de chave substituta (`id_token`) com hash sobre `moeda`, `nome_token` e `endereco_token`  
- Escrita final como `dim_token`, com granularidade no nível do contrato/token  

---

## Dimensão `dim_rede`

### Motivação

Cada transação está associada a uma blockchain (Ethereum, Polygon, etc.). Essa dimensão padroniza e codifica as redes envolvidas.

### Etapas realizadas

- Extração dos valores distintos da coluna `rede`  
- Geração do `id_rede` com hash da `rede` padronizada  
- Escrita da dimensão `dim_rede`, permitindo filtro e agregações por blockchain  

---

## Dimensão `dim_tipo_contrato`

### Motivação

Algumas transações referenciam o tipo de contrato e o token relacionado ao contrato inteligente (por exemplo, LP Tokens, NFTs ou contratos de DEX).

### Etapas realizadas

- Extração das colunas `tipo_contrato`, `simbolo_token_contrato` e `nome_token_contrato`  
- Remoção de nulos e duplicatas  
- Geração de `id_tipo_contrato` com base no hash dessas três informações combinadas  
- Escrita da dimensão `dim_tipo_contrato` com o objetivo de categorizar os contratos envolvidos  

---

## Boas Práticas Aplicadas

- Uso de funções determinísticas (`sha2`) para chaves substitutas  
- Padronização semântica de campos com `lower`, `trim`, `coalesce`  
- Eliminação de duplicidades e nulos antes da persistência  
- Escrita em Delta Lake, garantindo versionamento, performance e auditabilidade  
- Uso de alias descritivos e consistentes para todas as colunas  

---

## Próximos Passos

- Criação da tabela `fato_dex_transacoes` com chaves substitutas das dimensões  
- Enriquecimento da camada Silver com joins entre fatos e dimensões  
- Publicação de datasets analíticos com granularidade otimizada  
- Desenvolvimento de dashboards exploratórios para:  
  - Volume de transações por protocolo e token  
  - Gasto de gas por rede  
  - Participação de exchanges no mercado DEX  


# Criação da dimensão dim_calendario

In [0]:
from pyspark.sql.functions import (
    col, date_format, dayofmonth, month, year, dayofweek, when,
    create_map, lit, lower
)

# Pegando os dias distintos da Bronze
df_datas = spark.table("workspace.ethereum.dex_trades_bronze") \
    .select("data_transacao") \
    .dropna() \
    .distinct()

# Criando dicionário para ajustar os nomes dos meses e dos dias de inglês para português
mapa_dias = {
    "monday": "Segunda-feira",
    "tuesday": "Terça-feira",
    "wednesday": "Quarta-feira",
    "thursday": "Quinta-feira",
    "friday": "Sexta-feira",
    "saturday": "Sábado",
    "sunday": "Domingo"
}

mapa_meses = {
    "january": "Janeiro",
    "february": "Fevereiro",
    "march": "Março",
    "april": "Abril",
    "may": "Maio",
    "june": "Junho",
    "july": "Julho",
    "august": "Agosto",
    "september": "Setembro",
    "october": "Outubro",
    "november": "Novembro",
    "december": "Dezembro"
}

# Descobri que o create_map do Spark pode receber pares chave-valor diretamente como literais.
# Para isso, precisei transformar meu dicionário Python em uma sequência achatada com sum(d.items(), ()).
# Esse truque gera uma lista do tipo ("Monday", "Segunda-feira", "Tuesday", "Terça-feira", ...).
# Depois aplico lit() em cada item para transformar em expressão Spark, e passo para create_map.
# Com isso, consigo usar meu dicionário como expressão de mapeamento direto no withColumn

def dict_to_map_expr(d):
    return create_map([lit(x) for x in sum(d.items(), ())])

dias_map_expr = dict_to_map_expr(mapa_dias)
meses_map_expr = dict_to_map_expr(mapa_meses)

# Construindo a dimensão e convertendo nomes para português
dim_calendario = df_datas.withColumn("id_data", date_format("data_transacao", "yyyyMMdd").cast("int")) \
    .withColumn("data", col("data_transacao")) \
    .withColumn("dia", dayofmonth("data_transacao")) \
    .withColumn("mes", month("data_transacao")) \
    .withColumn("nome_mes_en", lower(date_format("data_transacao", "MMMM"))) \
    .withColumn("nome_mes", meses_map_expr[col("nome_mes_en")]) \
    .withColumn("ano", year("data_transacao")) \
    .withColumn("nome_dia_en", lower(date_format("data_transacao", "EEEE"))) \
    .withColumn("nome_dia", dias_map_expr[col("nome_dia_en")]) \
    .withColumn("fim_de_semana", when(dayofweek("data_transacao").isin(1, 7), True).otherwise(False)) \
    .drop("data_transacao", "nome_mes_en", "nome_dia_en")

# Salvando a dimensão de calendário no mesmo esquema da Bronze
dim_calendario.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("workspace.ethereum.dim_calendario")


# Criação da dimensã dim_exchange

In [0]:
from pyspark.sql.functions import sha2, concat_ws, col, lower, trim, coalesce, lit

# Selecionando campos distintos da bronze
df_exchange = spark.table("workspace.ethereum.dex_trades_bronze") \
    .select(
        col("exchange_nome").alias("nome_exchange"),
        col("exchange_nome_completo").alias("nome_completo"),
        col("protocolo")
    ).dropna().dropDuplicates()

# Criando um id com hash como surrogate key
#poderia usar outras técnicas mais avançadas como SCD Tipo 2, porém, para o projeto atual manterei a consistência total do hash garantindo que virá igual com coalesce, lowe, trim e lit
dim_exchange = df_exchange.withColumn(
    "id_exchange",
    sha2(
        concat_ws("||",
            coalesce(lower(trim(col("nome_exchange"))), lit("")),
            coalesce(lower(trim(col("nome_completo"))), lit("")),
            coalesce(lower(trim(col("protocolo"))), lit(""))
        ),
        256
    )
)

# Reordenando as colunas
dim_exchange = dim_exchange.select(
    "id_exchange", "nome_exchange", "nome_completo", "protocolo"
)

# Criando a dim_exchange

dim_exchange.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("workspace.ethereum.dim_exchange")


# Criando a dimensão dim_token

In [0]:
from pyspark.sql.functions import (
    col, lower, trim, coalesce, lit, concat_ws, sha2
)

df_token_base = spark.table("workspace.ethereum.dex_trades_bronze").select(
    col("moeda_base").alias("moeda"),
    col("nome_base").alias("nome_token"),
    col("endereco_base").alias("endereco_token")
)

df_token_quote = spark.table("workspace.ethereum.dex_trades_bronze").select(
    col("moeda_quote").alias("moeda"),
    col("nome_quote").alias("nome_token"),
    col("endereco_quote").alias("endereco_token")
)

# Linha pra unir os dois tipos de token que podem ser usados nesse tipo de operaão (conceito de mercado financeiro), quero uma dim com todos os tokens distintos, independentemennte se é a moeda base ou quote
df_token = df_token_base.union(df_token_quote).dropna().dropDuplicates()

dim_token = df_token.withColumn(
    "id_token",
    sha2(
        concat_ws("||",
            coalesce(lower(trim(col("moeda"))), lit("")),
            coalesce(lower(trim(col("nome_token"))), lit("")),
            coalesce(lower(trim(col("endereco_token"))), lit(""))
        ),
        256
    )
)

dim_token = dim_token.select(
    "id_token", "moeda", "nome_token", "endereco_token"
)

dim_token.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("workspace.ethereum.dim_token")


# Criando dimensão rede

In [0]:
from pyspark.sql.functions import col, sha2, lower, trim, coalesce, lit

df_rede = spark.table("workspace.ethereum.dex_trades_bronze") \
    .select("rede") \
    .dropna() \
    .dropDuplicates()

dim_rede = df_rede.withColumn(
    "id_rede",
    sha2(coalesce(lower(trim(col("rede"))), lit("")), 256)
)

dim_rede = dim_rede.select("id_rede", "rede")

dim_rede.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("workspace.ethereum.dim_rede")

# Criando dimensão dim_tipo_contrato

In [0]:
from pyspark.sql.functions import col, sha2, lower, trim, coalesce, concat_ws, lit

df_tipo_contrato = spark.table("workspace.ethereum.dex_trades_bronze") \
    .select(
        col("tipo_contrato"),
        col("simbolo_token_contrato").alias("simbolo_token"),
        col("nome_token_contrato").alias("nome_token")
    ) \
    .dropna(subset=["tipo_contrato", "nome_token"]) \
    .dropDuplicates()

dim_tipo_contrato = df_tipo_contrato.withColumn(
    "id_tipo_contrato",
    sha2(
        concat_ws("||",
            coalesce(lower(trim(col("tipo_contrato"))), lit("")),
            coalesce(lower(trim(col("simbolo_token"))), lit("")),
            coalesce(lower(trim(col("nome_token"))), lit(""))
        ), 256
    )
)

dim_tipo_contrato = dim_tipo_contrato.select(
    "id_tipo_contrato", "tipo_contrato", "simbolo_token", "nome_token"
)

dim_tipo_contrato.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("workspace.ethereum.dim_tipo_contrato")